# Feature extraction using LSTM (Original feature -> Improve feature)

---



Ở giai đoạn Feature extraction using LSTM, chúng ta sẽ sử dụng mô hình LSTM để trích xuất các đặc trưng từ dữ liệu đầu vào. Đầu ra của giai đoạn này sẽ là các đặc trưng đã được trích xuất, được biểu diễn dưới dạng một ma trận kích thước (số lượng mẫu, số chiều đặc trưng), trong đó mỗi hàng đại diện cho một mẫu dữ liệu và mỗi cột đại diện cho một đặc trưng.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
# Đọc tập dữ liệu
df = pd.read_csv('/content/drive/MyDrive/NT204_IDPS/UNSW-NB15_1_label.csv')

<ipython-input-9-807a0538a79e>:2: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/NT204_IDPS/UNSW-NB15_1_label.csv')


In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,59.166.0.8,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,NaN,0
699997,59.166.0.0,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,NaN,0
699998,59.166.0.0,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,NaN,0
699999,59.166.0.6,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,NaN,0


In [ ]:
# Chuyển đổi các giá trị categorical thành dạng số
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].cat.codes

In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,30,396,22,80621,120,2,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,-1,0
1,30,32659,25,60640,120,2,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,-1,0
2,36,470,23,80621,120,2,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,-1,0
3,35,2599,21,80621,120,2,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,-1,0
4,33,48659,6,80621,120,2,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,38,11525,22,28199,114,5,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,-1,0
699997,30,17900,25,9,114,5,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,-1,0
699998,30,29101,21,4009,114,5,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,-1,0
699999,36,29386,21,13,120,2,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,-1,0


In [ ]:
# Tách các cột dữ liệu và không lấy cột nhãn
data = df.iloc[:, :-1]

In [ ]:
data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
0,30,396,22,80621,120,2,0.001055,132,164,31,...,0,0,3,7,1,3,1,1,1,-1
1,30,32659,25,60640,120,2,0.036133,528,304,31,...,0,0,2,4,2,3,1,1,2,-1
2,36,470,23,80621,120,2,0.001119,146,178,31,...,0,0,12,8,1,2,2,1,1,-1
3,35,2599,21,80621,120,2,0.001209,132,164,31,...,0,0,6,9,1,1,1,1,1,-1
4,33,48659,6,80621,120,2,0.001169,146,178,31,...,0,0,7,9,1,1,1,1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,38,11525,22,28199,114,5,0.020383,320,1874,31,...,0,0,8,20,7,5,1,1,4,-1
699997,30,17900,25,9,114,5,1.402957,19410,1087890,31,...,0,0,1,1,2,7,2,2,2,-1
699998,30,29101,21,4009,114,5,0.007108,2158,2464,31,...,0,0,13,13,6,7,2,1,2,-1
699999,36,29386,21,13,120,2,0.004435,568,304,31,...,0,0,10,13,6,5,1,1,3,-1


In [ ]:
data.shape

(700001, 48)

In [ ]:
hidden_size = 64 # Số lượng đơn vị ẩn trong mỗi lớp LSTM
output_size = 48 # Số đầu ra mong muốn 

''' Xây dựng mô hình Bi-LSTM'''
model_lstm = tf.keras.Sequential()
# Thêm lớp LSTM chuyển tiếp ở phía trước
model_lstm.add(tf.keras.layers.LSTM(units=hidden_size, return_sequences=True, input_shape=(data.shape[1], 1))) # 48 đầu vào 
# Thêm lớp LSTM chuyển tiếp ở phía sau với tham số go_backwards=True
model_lstm.add(tf.keras.layers.LSTM(units=hidden_size, return_sequences=False, go_backwards=True))
# Thêm lớp đầu ra
model_lstm.add(tf.keras.layers.Dense(output_size, activation='softmax')) # Trích xuất 48 đầu ra từ 48 đầu vào 
# Compile mô hình
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Trong đoạn mã trên, **hidden_size** đại diện cho số lượng đơn vị ẩn trong mỗi lớp LSTM, **output_size** là số lượng lớp đầu ra trong bài toán và **activation** là hàm kích hoạt được áp dụng cho lớp đầu ra. 

Ta đã đặt tham số **return_sequences=False** cho lớp LSTM cuối cùng ở phía sau (backward LSTM). Điều này đảm bảo rằng lớp cuối cùng này chỉ trả về đầu ra của phía sau và không truyền chuỗi đầu ra tới lớp đầu ra.

Lưu ý rằng lớp LSTM ở phía trước (forward LSTM) vẫn sử dụng **return_sequences=True**, giúp truyền chuỗi đầu ra tới lớp LSTM ở phía sau. Tuy nhiên, vì lớp LSTM cuối cùng chỉ quan tâm đến đầu ra của phía trước, chúng ta đặt **return_sequences=False** cho lớp LSTM cuối cùng ở phía sau để chỉ trả về một vector đầu ra duy nhất.

Qua đó, mô hình LSTM hai chiều giúp tận dụng dữ liệu từ cả hai phía của chuỗi đầu vào và nắm bắt được các mẫu và mối quan hệ quan trọng trong dữ liệu, đem lại khả năng mô hình hóa và dự đoán tốt hơn trong nhiều bài toán xử lý chuỗi dữ liệu.

In [ ]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 48, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 48)                3120      
                                                                 
Total params: 53,040
Trainable params: 53,040
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Output of LSTM model 
lstm_output = model_lstm.predict(data)

21876/21876 [==============================] - 478s 22ms/step


In [ ]:
lstm_output

array([[0.02431011, 0.02442381, 0.02307024, ..., 0.0188068 , 0.02439165,
        0.02847754],
       [0.02438628, 0.02462359, 0.02321933, ..., 0.01897505, 0.02451356,
        0.02866396],
       [0.02435596, 0.02451459, 0.02299988, ..., 0.01888407, 0.02456644,
        0.02844921],
       ...,
       [0.02443561, 0.02494098, 0.02317226, ..., 0.01880102, 0.02465041,
        0.02893801],
       [0.02380369, 0.02419267, 0.02323011, ..., 0.01939099, 0.02373918,
        0.0286246 ],
       [0.02442384, 0.02497632, 0.02308998, ..., 0.01866931, 0.02463741,
        0.02900439]], dtype=float32)

In [ ]:
lstm_output.shape

(700001, 48)

In [ ]:
# Save features to DataFrame 
features_lstm = pd.DataFrame(lstm_output)

In [ ]:
features_lstm

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0.024310,0.024424,0.023070,0.019863,0.016562,0.018703,0.025806,0.028078,0.017529,0.023081,...,0.022306,0.023913,0.021791,0.019090,0.015523,0.020514,0.016835,0.018807,0.024392,0.028478
1,0.024386,0.024624,0.023219,0.019578,0.016578,0.018586,0.025689,0.027998,0.017569,0.023082,...,0.022235,0.023899,0.021869,0.019171,0.015670,0.020248,0.016877,0.018975,0.024514,0.028664
2,0.024356,0.024515,0.023000,0.019837,0.016388,0.018799,0.025967,0.028125,0.017655,0.023027,...,0.022204,0.023965,0.021878,0.019166,0.015525,0.020473,0.016794,0.018884,0.024566,0.028449
3,0.024254,0.024496,0.023112,0.019680,0.016528,0.018680,0.025742,0.028053,0.017606,0.023021,...,0.022235,0.023980,0.021923,0.019215,0.015608,0.020358,0.016719,0.019120,0.024422,0.028541
4,0.024228,0.024329,0.023561,0.019192,0.017079,0.018315,0.025884,0.027892,0.017333,0.022921,...,0.022213,0.023672,0.021903,0.019370,0.015543,0.020272,0.016264,0.019594,0.024202,0.028077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.024437,0.024879,0.023097,0.019503,0.016232,0.018553,0.025983,0.028032,0.017536,0.023189,...,0.022227,0.023720,0.021957,0.019199,0.015747,0.020274,0.016802,0.018938,0.024783,0.028732
699997,0.023952,0.024550,0.023278,0.019605,0.016745,0.018272,0.025519,0.027755,0.017118,0.023416,...,0.022785,0.023231,0.021751,0.019043,0.015831,0.020511,0.016597,0.018997,0.024032,0.028649
699998,0.024436,0.024941,0.023172,0.019440,0.016329,0.018346,0.025762,0.027961,0.017331,0.023398,...,0.022379,0.023600,0.021887,0.019149,0.015829,0.020282,0.016847,0.018801,0.024650,0.028938
699999,0.023804,0.024193,0.023230,0.019923,0.016865,0.018628,0.025228,0.027937,0.017463,0.023085,...,0.022730,0.023745,0.021949,0.019289,0.015722,0.020485,0.016573,0.019391,0.023739,0.028625


In [ ]:
# Tạo một Series mới chứa dữ liệu cho cột mới
label_col = df.iloc[:, -1]

# Thêm cột mới vào DataFrame bằng cách gán giá trị cho loc với tên cột mới
features_lstm.loc[:, 'Label'] = label_col

In [ ]:
features_lstm

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,Label
0,0.024310,0.024424,0.023070,0.019863,0.016562,0.018703,0.025806,0.028078,0.017529,0.023081,...,0.023913,0.021791,0.019090,0.015523,0.020514,0.016835,0.018807,0.024392,0.028478,0
1,0.024386,0.024624,0.023219,0.019578,0.016578,0.018586,0.025689,0.027998,0.017569,0.023082,...,0.023899,0.021869,0.019171,0.015670,0.020248,0.016877,0.018975,0.024514,0.028664,0
2,0.024356,0.024515,0.023000,0.019837,0.016388,0.018799,0.025967,0.028125,0.017655,0.023027,...,0.023965,0.021878,0.019166,0.015525,0.020473,0.016794,0.018884,0.024566,0.028449,0
3,0.024254,0.024496,0.023112,0.019680,0.016528,0.018680,0.025742,0.028053,0.017606,0.023021,...,0.023980,0.021923,0.019215,0.015608,0.020358,0.016719,0.019120,0.024422,0.028541,0
4,0.024228,0.024329,0.023561,0.019192,0.017079,0.018315,0.025884,0.027892,0.017333,0.022921,...,0.023672,0.021903,0.019370,0.015543,0.020272,0.016264,0.019594,0.024202,0.028077,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.024437,0.024879,0.023097,0.019503,0.016232,0.018553,0.025983,0.028032,0.017536,0.023189,...,0.023720,0.021957,0.019199,0.015747,0.020274,0.016802,0.018938,0.024783,0.028732,0
699997,0.023952,0.024550,0.023278,0.019605,0.016745,0.018272,0.025519,0.027755,0.017118,0.023416,...,0.023231,0.021751,0.019043,0.015831,0.020511,0.016597,0.018997,0.024032,0.028649,0
699998,0.024436,0.024941,0.023172,0.019440,0.016329,0.018346,0.025762,0.027961,0.017331,0.023398,...,0.023600,0.021887,0.019149,0.015829,0.020282,0.016847,0.018801,0.024650,0.028938,0
699999,0.023804,0.024193,0.023230,0.019923,0.016865,0.018628,0.025228,0.027937,0.017463,0.023085,...,0.023745,0.021949,0.019289,0.015722,0.020485,0.016573,0.019391,0.023739,0.028625,0


# Feature matrix - Ma trận đặc trưng (Tìm kiếm các đặc trưng quan trọng trong dữ liệu)

In [ ]:
# Extract the features from the DataFrame
features = features_lstm.drop('Label', axis=1).to_numpy()
# Extract the labels from the DataFrame
labels = features_lstm['Label'].to_numpy()

In [ ]:
features, labels

(array([[0.02431011, 0.02442381, 0.02307024, ..., 0.0188068 , 0.02439165,
         0.02847754],
        [0.02438628, 0.02462359, 0.02321933, ..., 0.01897505, 0.02451356,
         0.02866396],
        [0.02435596, 0.02451459, 0.02299988, ..., 0.01888407, 0.02456644,
         0.02844921],
        ...,
        [0.02443561, 0.02494098, 0.02317226, ..., 0.01880102, 0.02465041,
         0.02893801],
        [0.02380369, 0.02419267, 0.02323011, ..., 0.01939099, 0.02373918,
         0.0286246 ],
        [0.02442384, 0.02497632, 0.02308998, ..., 0.01866931, 0.02463741,
         0.02900439]], dtype=float32),
 array([0, 0, 0, ..., 0, 0, 0]))

In [ ]:
''' Normalize the features (chuẩn hóa các đặc trưng (features) của dữ liệu) '''
# np.mean(features, axis=0): Đây là hàm tính trung bình theo trục 0 (theo cột) của ma trận đặc trưng (features).
# np.std(features, axis=0): Đây là hàm tính độ lệch chuẩn theo trục 0 (theo cột) của ma trận đặc trưng (features)
# features - np.mean(features, axis=0): Kết quả là một ma trận có cùng kích thước với features, trong đó giá trị của mỗi ô được điều chỉnh theo độ lệch so với trung bình của đặc trưng đó
# (features - np.mean(features, axis=0)) / np.std(features, axis=0): Bước cuối cùng là chia kết quả từ bước 3 cho độ lệch chuẩn của từng đặc trưng. Điều này chuẩn hóa giá trị của mỗi đặc trưng để có phân phối có trung bình bằng 0 và độ lệch chuẩn bằng 1.
normalized_features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

In [ ]:
normalized_features

array([[ 0.5458861 , -0.4981182 , -1.0202378 , ..., -0.7080724 ,
         0.30945715, -0.3319252 ],
       [ 0.7252963 , -0.0795542 , -0.40193084, ..., -0.36082712,
         0.5234761 ,  0.01489061],
       [ 0.6538718 , -0.3079335 , -1.312007  , ..., -0.54860055,
         0.6163244 , -0.38464013],
       ...,
       [ 0.84147257,  0.58544636, -0.5971568 , ..., -0.7200089 ,
         0.76373774,  0.5247408 ],
       [-0.6468658 , -0.98240954, -0.35721263, ...,  0.4976178 ,
        -0.8360432 , -0.05833919],
       [ 0.813738  ,  0.65947837, -0.93837935, ..., -0.99183726,
         0.74091566,  0.64823556]], dtype=float32)

In [ ]:
# Create the feature matrix with labels
feature_matrix = np.hstack((normalized_features, labels.reshape(-1, 1)))

In [ ]:
feature_matrix, feature_matrix.shape

(array([[ 0.5458861 , -0.49811819, -1.0202378 , ...,  0.30945715,
         -0.33192521,  0.        ],
        [ 0.72529632, -0.0795542 , -0.40193084, ...,  0.52347612,
          0.01489061,  0.        ],
        [ 0.65387177, -0.30793351, -1.31200695, ...,  0.61632442,
         -0.38464013,  0.        ],
        ...,
        [ 0.84147257,  0.58544636, -0.59715682, ...,  0.76373774,
          0.52474082,  0.        ],
        [-0.64686579, -0.98240954, -0.35721263, ..., -0.83604318,
         -0.05833919,  0.        ],
        [ 0.81373799,  0.65947837, -0.93837935, ...,  0.74091566,
          0.64823556,  0.        ]]),
 (700001, 49))

# Chia features và labels cho bài toán phân loại (Binary Classification)

In [ ]:
X_cnn = feature_matrix[:, :-1] # Trích xuất dữ liệu (features)
y_cnn = feature_matrix[:, -1] # Trích xuất nhãn (labels)

In [ ]:
X_cnn, X_cnn.shape

(array([[ 0.5458861 , -0.49811819, -1.0202378 , ..., -0.70807242,
          0.30945715, -0.33192521],
        [ 0.72529632, -0.0795542 , -0.40193084, ..., -0.36082712,
          0.52347612,  0.01489061],
        [ 0.65387177, -0.30793351, -1.31200695, ..., -0.54860055,
          0.61632442, -0.38464013],
        ...,
        [ 0.84147257,  0.58544636, -0.59715682, ..., -0.72000891,
          0.76373774,  0.52474082],
        [-0.64686579, -0.98240954, -0.35721263, ...,  0.49761781,
         -0.83604318, -0.05833919],
        [ 0.81373799,  0.65947837, -0.93837935, ..., -0.99183726,
          0.74091566,  0.64823556]]),
 (700001, 48))

In [ ]:
y_cnn, y_cnn.shape

(array([0., 0., 0., ..., 0., 0., 0.]), (700001,))

In [ ]:
# Reshape dữ liệu để phù hợp với mô hình CNN
X_cnn = X_cnn.reshape(X_cnn.shape[0], X_cnn.shape[1], 1)

In [ ]:
X_cnn.shape

(700001, 48, 1)

In [ ]:
# Convert array thành DataFrame
y_cnn = pd.DataFrame(y_cnn, columns=['Class'])
# sử dụng pd.get_dummies() để reshape cột "Class"
y_cnn = pd.get_dummies(y_cnn, columns=['Class']).values

In [ ]:
y_cnn

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

# Train by CNN

In [ ]:
from sklearn.model_selection import train_test_split
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 7:3
X_train, X_test, y_train, y_test = train_test_split(X_cnn, y_cnn, test_size=0.3, random_state=0)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Dropout, LSTM, MaxPooling1D, Flatten
from keras.models import Sequential
# Xây dựng mô hình CNN
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Dropout(0.2))
model_cnn.add(Flatten())
model_cnn.add(Dense(units=2, activation='softmax')) # obtains the output probability of each neuron and its fully connected layer weight

In [ ]:
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 46, 64)            256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 64)            0         
                                                                 
 flatten (Flatten)           (None, 1472)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2946      
                                                                 
Total params: 3,202
Trainable params: 3,202
Non-trainable params: 0
____________________________________________________

In [ ]:
# Compile the model
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Huấn luyện mô hình
model_cnn.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
3829/3829 [==============================] - 62s 16ms/step - loss: 0.0314 - accuracy: 0.9885 - val_loss: 0.0176 - val_accuracy: 0.9930
Epoch 2/5
3829/3829 [==============================] - 66s 17ms/step - loss: 0.0185 - accuracy: 0.9925 - val_loss: 0.0155 - val_accuracy: 0.9944
Epoch 3/5
3829/3829 [==============================] - 66s 17ms/step - loss: 0.0169 - accuracy: 0.9932 - val_loss: 0.0146 - val_accuracy: 0.9943
Epoch 4/5
3829/3829 [==============================] - 62s 16ms/step - loss: 0.0161 - accuracy: 0.9933 - val_loss: 0.0139 - val_accuracy: 0.9945
Epoch 5/5
3829/3829 [==============================] - 62s 16ms/step - loss: 0.0154 - accuracy: 0.9936 - val_loss: 0.0137 - val_accuracy: 0.9946


# Weight set => Probability

Xem dòng **model_cnn.add(Dense(units=2, activation='softmax'))**

The activation function obtains the output probability of each neuron and its fully connected layer weight. 
Generally, the value of activation function lies between 0 to 1 and a maximum value represents the output label. 

# Intrusion detection by CNN (Results)

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_acc = model_cnn.evaluate(X_test, y_test)
print('Test lost:', test_loss)
print('Test accuracy:', test_acc)

6563/6563 [==============================] - 17s 3ms/step - loss: 0.0137 - accuracy: 0.9946
Test lost: 0.013689273968338966
Test accuracy: 0.9946286082267761


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Predict on test set
y_pred_test = model_cnn.predict(X_test)

print(classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), digits=4))

# Compute the confusion matrix
cm = confusion_matrix(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
print(cm)

6563/6563 [==============================] - 14s 2ms/step
              precision    recall  f1-score   support

           0     0.9984    0.9961    0.9972    203300
           1     0.8883    0.9514    0.9187      6701

    accuracy                         0.9946    210001
   macro avg     0.9433    0.9737    0.9580    210001
weighted avg     0.9949    0.9946    0.9947    210001

[[202498    802]
 [   326   6375]]


In [ ]:
# Predict on X_cnn set để đưa ra dự đoán cho tất cả các request
y_pred_total = model_cnn.predict(X_cnn)

21876/21876 [==============================] - 49s 2ms/step


In [ ]:
y_pred_total[10000][0], y_pred_total[10000][1]

(0.99999994, 7.234812e-23)

In [ ]:
number_of_normal = 0
number_of_attack = 0
def final_decision_IDS(index):
  if (y_pred_total[index][0] > y_pred_total[index][1]):
    return "Normal"
  else:
    return "Attack"

In [ ]:
number_of_packets = len(df)

In [ ]:
number_of_packets

700001

In [ ]:
for i in range(number_of_packets):
  print("Final classification for packet {} is {}".format(i, final_decision_IDS(i)))
  if final_decision_IDS(i) == "Normal":
    number_of_normal += 1
  else:
    number_of_attack += 1

Streaming output truncated to the last 5000 lines.
Final classification for packet 695001 is Normal
Final classification for packet 695002 is Normal
Final classification for packet 695003 is Normal
Final classification for packet 695004 is Normal
Final classification for packet 695005 is Normal
Final classification for packet 695006 is Normal
Final classification for packet 695007 is Normal
Final classification for packet 695008 is Normal
Final classification for packet 695009 is Normal
Final classification for packet 695010 is Normal
Final classification for packet 695011 is Normal
Final classification for packet 695012 is Normal
Final classification for packet 695013 is Normal
Final classification for packet 695014 is Normal
Final classification for packet 695015 is Normal
Final classification for packet 695016 is Normal
Final classification for packet 695017 is Normal
Final classification for packet 695018 is Normal
Final classification for packet 695019 is Normal
Final classificati

In [ ]:
print("Number of Normal packets: ", number_of_normal)

Number of Normal packets:  676113


In [ ]:
print("Number of Attack packets: ", number_of_attack)

Number of Attack packets:  23888
